# Tabular Learning and the Bellman Equation

In the previous chapter, we got acquainted with our first Reinforcement Learning (**RL**) method, **cross-entropy**, and saw its strengths and weaknesses. In this new part of the book, we'll look at another group of methods, called **Q-learning**, which have much more flexibility and power.
This chapter will establish the required background shared by those methods. We'll also revisit the FrozenLake environment and show how new concepts will fit with this environment and help us to address the issues of the environment's uncertainty.

## Main Terms


### Belman Equation of Value - Determinsitc Case

![](img/Ch5-Fig3.png)


The Value of state $S_0$ is equal to the maximum of the sum of the possible rewards and the value of the next step over all possible actions. So, to choose the best possible action, the agent needs to calculate the resulting values for every action and choose the maximum possible outcome. This is a recursive claulation.

$V_0 = \max\in_{1...N}(r_a + \gamma V_a)$ **Belman Equation**

Where:
* $\gamma$ : discount value
* $r_a$: reward of action $a$
* $V_a$: value of next state given action $a$

### Belman Equation of Value - Stochastic Case

![](img/Ch5-Fig4.png)

$V_0 = \max_{a \in A}\mathbb{E}_{s~S}(r_{s,a} + \gamma V_s) = \max_{a \in A} \sum_{s \in S}p_{a,0 \rightarrow s}(r_{s,a} + \gamma V_s)$   

Where:
* $p_{a,0 \rightarrow s}$: probability of action $a$, issued in state $i$, to end up in state $j$

### Value of action

Value of action is equals the total reward we can get by executing action $a$ in state $s$ and can be defined via $V_s$. This quantity gave a name to the whole family of methods called **"Q-learning"**, because it is slightly more convenient in practice. In these methods, our primary objective is to get values of $Q$ for every pair of state and action. 
The Q value is:

$Q_{s,a} = \mathbb{E}_{s'~S}(r_{s,a} + \gamma V_{s'}) = \sum_{s \in S}p_{a,s \rightarrow s'}(r_{s,a} + \gamma V_{s'})$   

$Q$ for this state $s$ and action a equals the expected immediate reward and the discounted long-term reward of the destination state $s'$. 
We also can define $V_s$ via $Q_{s,a}$:

$V_s = \max_{a \in A}Q_{s,a}$

This just means that the value of some state equals to the value of the maximum action we can execute from this state. It may look very close to the value of state, but there is still a difference, which is important to understand. Finally, we can express $Q(s, a)$ via itself, which will be used in the next chapter's topic of **Q-learning**:

$Q(s, a) = r_{s,a} + \gamma \max_{a' \in A}Q(s',a')$


### Value Integration Algorithm

The  **"value iteration algorithm"** allows us to numerically calculate the values of states and values of actions of MDPs with known transition probabilities and rewards. 

The procedure (for **values of states**) includes the following steps:
1. Initialize values of all states Vi to some initial value (usually zero) 
2. For every state s in the MDP, perform the Bellman update:
$$Vs \leftarrow \max_{a \in A} \sum_{s'}p_{a,s \rightarrow s'}(r_{s,a} + \gamma V_s')$$
3. Repeat step 2 for some large number of steps or until changes become too small

For f action values (**Q-Value**), only minor modifications to the preceding procedure are required:
1. Initialize all Qs,a to zero 
2. For every state s and every action a in this state, perform update: 
$$Q_{s,a} \leftarrow \sum_{s'}p_{a,s \rightarrow s'}(r_{s,a} + \gamma \max_{a'} Q_{s',a'})$$
3. Repeat step 2

### Value iteration in practice

The central data structures in this example are as follows: 
* **Reward table**: A dictionary with the composite key "source state" + "action" + "target state". The value is obtained from the immediate reward.
* **Transitions table**: A dictionary keeping counters of the experienced transitions. The key is the composite "state" + "action" and the value is another dictionary that maps the target state into a count of times that we've seen it. For example, if in state 0 we execute action 1 ten times, after three times it leads us to state 4 and after seven times to state 5. Entry with the key (0, 1) in this table will be a dict {4: 3, 5: 7}. We use this table to estimate the probabilities of our transitions.
* **Value table**: A dictionary that maps a state into the calculated value of this state.

The overall logic of our code is simple: in the loop, we play 100 random steps from the environment, populating the reward and transition tables. After those 100 steps, we perform a value iteration loop over all states, updating our value table. Then we play several full episodes to check our improvements using the updated value table. If the average reward for those test episodes is above the 0.8 boundary, then we stop training. During test episodes, we also update our reward and transition tables to use all data from the environment.

First the Agent class code, which will keep our tables and contain functions we'll be using in the training loop:


In [235]:
import gym
import collections
from tensorboardX import SummaryWriter

ENV_NAME = "FrozenLake-v0"
#ENV_NAME = "FrozenLake8x8-v0"

class Agent:
    def __init__(self): #Init method for the class
        self.env = gym.make(ENV_NAME) #Creating a Gym environment
        self.state = self.env.reset() #Reseting the environment - state stores the current state
        self.rewards = collections.defaultdict(float) #Creating a dictionary for rewards table
        #Examplae {(0, 0, 0): 0.0, - key: ("source state","action","target state"), Value: immediate reward
        #          (0, 3, 1): 0.0}
        self.transits = collections.defaultdict(collections.Counter) #Creating a dictionary for tranists table
        #{(0, 0): Counter({0: 1947, 4: 947}), Key: ("state", "action"), Value: dictionary that with key: next state, value: count of times that we've seen it
        # (0, 3): Counter({1: 112, 0: 172})}
        self.values = collections.defaultdict(float) #creating a dictionary for values
        #{0: 0.06752900040833315, Key: State, Value - Clauclated value of the state
        # 4: 0.1010535385685904,
        # 1: 0.06666611422673593}
        
    #Method to play n random stpes - n = count    
    #This function is used to gather random experience from the environment and update reward and transition tables
    def play_n_random_steps(self, count): 
        for _ in range(count):
            action = self.env.action_space.sample() #get a sample from the action space frmo the environment (the sample is conducted by the environment)
            new_state, reward, is_done, _ = self.env.step(action)  #make a step with the given action
            self.rewards[(self.state, action, new_state)] = reward #store the returned reward in the rewards dictionery
            self.transits[(self.state, action)][new_state] += 1 #increment the counter in the transits table
            self.state = self.env.reset() if is_done else new_state #reste the envirment if n steps were taken
            #print('Curren state=', self.state)
        self.value_iteration() #populate the the value table
    
    #method to claculate the action,value (Q-Value) for a given state and action
    #function calculates the value of the action from the state, using our transition, reward and values tables
    def calc_action_value(self, state, action):
        target_counts = self.transits[(state, action)] #get all coutns for all next states
        total = sum(target_counts.values()) #sum all the counts into tooal
        action_value = 0.0
        for tgt_state, count in target_counts.items(): #iterate over all next states 
            reward = self.rewards[(state, action, tgt_state)] #obtain the reward for a given sata, action and next state 
            #implementation of the Bellman Equation
            prob = (count / total)
            action_value +=  prob * (reward + GAMMA * self.values[tgt_state]) #incrmenting the action_value using Bellman equation = sum(prob*(r + gamma * Vs))
        return action_value
    
    #method to select the best action  in a given state
    #The decision is done in a greedy way (exploit only, no explore)
    #This action selection process is deterministic, as the play_n_random_steps() function introduces enough exploration.
    def select_action(self, state): 
        best_action, best_value = None, None
        for action in range(self.env.action_space.n): #iterate over all possible actions
            action_value = self.calc_action_value(state, action) #calculate the action value (Q-Value)
            if best_value is None or best_value < action_value: #calulate the best value
                best_value = action_value
                best_action = action
        return best_action
    
    #method to play a single episode 
    #Find the best action to take and plays one full episode using the provided environment
    #It makes use of test env and not the main environment 
    def play_episode(self, env): 
        total_reward = 0.0 #reseting total rewards
        state = env.reset() #reset the environment 
        while True:
            action = self.select_action(state) #Call method to select the best action (against the policy) 
            new_state, reward, is_done, _ = env.step(action) #make a step on the enviroment 
            self.rewards[(state, action, new_state)] = reward #store the returned reward in the rewards table/dictionery 
            self.transits[(state, action)][new_state] += 1 #increment the counter in the transits table
            total_reward += reward #increment total reward with the reward 
            #print('state=',new_state, ' reward=reward', ' total reward=', total_reward)
            if is_done: #if episode is completed
                break
            state = new_state
        return total_reward

    def value_iteration(self): #method to alculate and poulate the values table
        for state in range(self.env.observation_space.n): #iterate over all states
            state_values = [self.calc_action_value(state, action) #obtain the action value
                            for action in range(self.env.action_space.n)] #for each action
            self.values[state] = max(state_values) #store the max of state_values as the State's value





In [227]:
ENV_NAME

'FrozenLake8x8-v0'

The two lines in the preceding code snippet are the key piece in the training loop. First, we perform 100 random steps to fill our reward and transition tables with fresh data and then we run value iteration over all states. The rest of the code plays test episodes **using the value table as our policy**, then writes data into TensorBoard, tracks the best average reward, and checks for the training loop stop condition.

### Main loop
Th is thte main loop to call the intial exploration (play_n_random_steps()) and then playnumer of test train episodes.

In [242]:

GAMMA = 0.95 #Reward discount rate
TRAIN_EPISODES = 20 #mumber of test episodes
EXPLORE_STEPS = 100 #number of intial environment exlpore steps

if __name__ == "__main__":
    
    #Intializing env, agent and writer
    test_env = gym.make(ENV_NAME) #crating an instanse of the environment for testing (training?)
    agent = Agent() #creating an object instance of the agent
    writer = SummaryWriter(comment="-v-iteration") #cratring an tesnorflow object

    iter_no = 0 #iteration counter
    best_reward = 0.0
    #               Training loop
    #===============================
    while True:
        iter_no += 1
        agent.play_n_random_steps(EXPLORE_STEPS) #Play random 100 steps 
        #agent.value_iteration() #populate the the value table
 
        reward = 0.0
        for _ in range(TRAIN_EPISODES):
            reward += agent.play_episode(test_env)
        reward /= TRAIN_EPISODES
        writer.add_scalar("reward", reward, iter_no)
        if reward > best_reward:
            print("Best reward updated %.3f -> %.3f" % (best_reward, reward))
            best_reward = reward
        if reward > 0.80:
            print("Solved in %d iterations!" % iter_no)
            break
    writer.close()

Best reward updated 0.000 -> 0.200
Best reward updated 0.200 -> 0.250
Best reward updated 0.250 -> 0.350
Best reward updated 0.350 -> 0.400
Best reward updated 0.400 -> 0.700
Best reward updated 0.700 -> 0.750
Best reward updated 0.750 -> 0.800
Best reward updated 0.800 -> 0.850
Solved in 45 iterations!


#### Testing the policy
Using the trained policy above, we testing it on new epizods

In [243]:
TEST_EPISODES = 1000
env = gym.make(ENV_NAME) #Create a new environment
total_reward = 0
for i in range(TEST_EPISODES):
    rewards = agent.play_episode(env)
    #print('Episode: ',i,' Reward=', reward)
    total_reward += rewards
print('Average reward=', total_reward/TEST_EPISODES)
    

Average reward= 0.627


## Q-learning for FrozenLake 
The difference relatievly to ``Value Iteration`` above is really minor. The most obvious change is to the value table. In the previous example, we kept the value of the state, so the key in the dictionary was just a state. Now we need to store values of the Q-function, which has two parameters: state and action, so the key in the value table is now a composite.

The second difference is in the calc_action_value function. We just don't need it anymore, as our action values are stored in the value table. Finally, the most important change in the code is in the agent's value_iteration method. Before, it was just a wrapper around the calc_action_value call, which did the job of Bellman approximation. Now, as this function has gone and was replaced by a value table, we need to do this approximation in the value_iteration method.

In [1]:
import gym
import collections
from tensorboardX import SummaryWriter

ENV_NAME = "FrozenLake-v0"
#ENV_NAME = "FrozenLake8x8-v0"
GAMMA = 0.9
TEST_EPISODES = 10
EXPLORE_STEPS = 200 #number of intial environment exlpore steps


class Agent:
    def __init__(self):
        self.env = gym.make(ENV_NAME) #Creating a Gym environment
        self.state = self.env.reset() #Reseting the environment - state stores the current state
        self.rewards = collections.defaultdict(float) #Creating a dictionary for rewards table
        #Examplae {(0, 0, 0): 0.0, - key: ("source state","action","target state"), Value: immediate reward
        #          (0, 3, 1): 0.0}
        self.transits = collections.defaultdict(collections.Counter) #Creating a dictionary for tranists table
        #Transites table example:
        #{(0, 0): Counter({0: 1947, 4: 947}), Key: ("state", "action"), Value: dictionary that with key: next state, value: count of times that we've seen it
        # (0, 3): Counter({1: 112, 0: 172})}
        self.values = collections.defaultdict(float)
        #VAlue table example
        # {(4, 0): 0.09062526696040417, Key: (State, action), Value: Caluclated value of the state, value pairs
        #  (4, 1): 0.06728916471587301,
        #  (4, 2): 0.06721133905626454,

    #Method to play n random stpes - n = count    
    #This function is used to gather random experience from the environment and update reward and transition tables
    def play_n_random_steps(self, count):
        for _ in range(count):
            action = self.env.action_space.sample() #get a sample from the action space frmo the environment (the sample is conducted by the environment)
            new_state, reward, is_done, _ = self.env.step(action) #make a step with the given action
            self.rewards[(self.state, action, new_state)] = reward #store the returned reward in the rewards dictionery
            self.transits[(self.state, action)][new_state] += 1 #increment the counter in the transits table
            self.state = self.env.reset() if is_done else new_state  #reset the envirment if n steps were taken
            
    #method to select the best action  in a given state
    #The decision is done in a greedy way (exploit only, no explore)
    #This action selection process is deterministic, as the play_n_random_steps() function introduces enough exploration.             
    def select_action(self, state):
        best_action, best_value = None, None
        for action in range(self.env.action_space.n): #iterate over all possible actions
            action_value = self.values[(state, action)] #Get the action value (Q-Value)
            if best_value is None or best_value < action_value:  #calulate the best value
                best_value = action_value
                best_action = action
        return best_action
    
    #method to play a single episode 
    #Find the best action to take and plays one full episode using the provided environment
    #It makes use of test env and not the main environment 
    def play_episode(self, env):
        total_reward = 0.0  #reseting total rewards
        step_count = 0
        state = env.reset() #reset the environment 
        while True:
            action = self.select_action(state) #Call method to select the best action (against the policy) 
            new_state, reward, is_done, _ = env.step(action) #make a step on the enviroment 
            self.rewards[(state, action, new_state)] = reward #store the returned reward in the rewards table/dictionery 
            self.transits[(state, action)][new_state] += 1 #increment the counter in the transits table
            total_reward += reward #increment total reward with the new reward 
            step_count += 1
            if is_done: #if episode is completed
                break
            
            #print('current state:',state, 'action:',action,' new state', new_state, )
            state = new_state
        #print('==================== end of episode ==================')
        return total_reward, step_count

    #method to alculate and poulate the values table
    def value_iteration(self):
        for state in range(self.env.observation_space.n): #iterate over all states
            for action in range(self.env.action_space.n):
                action_value = 0.0 #reset action value pair
                target_counts = self.transits[(state, action)] #get all current counts of all state action paris from the tranists table
                total = sum(target_counts.values()) #summ all current counts to get total
                for tgt_state, count in target_counts.items(): #iterate over all target states and their correspending counts
                    reward = self.rewards[(state, action, tgt_state)]  #get the stored reward of the state, action, tgt_state from the rewards table 
                    best_action = self.select_action(tgt_state) #get the best action for the target state 
                    action_value += (count / total) * (reward + GAMMA * self.values[(tgt_state, best_action)]) #use Bellman equation to calculate the new action value
                self.values[(state, action)] = action_value #store the new calculated action value in the value table

if __name__ == "__main__":
    test_env = gym.make(ENV_NAME)
    agent = Agent()
    writer = SummaryWriter(comment="-q-iteration")

    iter_no = 0
    best_reward = 0.0
    step_count = 0
    episode_count = 0
    while True:
        iter_no += 1
        agent.play_n_random_steps(EXPLORE_STEPS) #play random steps
        agent.value_iteration() #update the vaue table - not sure why this is an exposed method

        reward = 0.0
        for _ in range(TEST_EPISODES):
            this_episode_reward, steps = agent.play_episode(test_env) #play a test episode - however this method update the reward and the transit tables
            reward += this_episode_reward
            step_count += steps
            
        reward /= TEST_EPISODES
        episode_count += TEST_EPISODES
        writer.add_scalar("reward", reward, iter_no)
        if reward > best_reward:
            print("Best reward updated %.3f -> %.3f" % (best_reward, reward))
            best_reward = reward
        if reward > 0.80:
            print("Solved in %d iterations!" % iter_no, ' test episodes = ', episode_count, ' total steps =', step_count, ' average steps per episode =', step_count/episode_count )
            break
    writer.close()


current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 4
current state: 4 action: 0  new state 4
current state: 4 action: 0  new state 4
current state: 4 action: 0  new state 8
current state: 8 action: 0  new state 8
current state: 8 action: 0  new state 8
current state: 8 action: 0  new state 4
current state: 4 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 4
current state: 4 action: 0  new state 8
current state: 8 action: 0  new state 8
current state: 8 action: 0  new state 4
current state: 4 action: 0  new state 4
current state: 4 action: 0  new state 8
current state: 8 action: 0  new state 4
current state: 4 action: 0  new state 8
==================== end of episode ==================
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 4
current state: 4 action: 0  new state 0
current state: 0 action: 0  new state 4
current state: 4 action: 

current state: 0 action: 0  new state 4
current state: 4 action: 0  new state 8
==================== end of episode ==================
current state: 0 action: 0  new state 4
current state: 4 action: 0  new state 4
current state: 4 action: 0  new state 4
current state: 4 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 4
current state: 4 action: 0  new state 8
current state: 8 action: 0  new state 8
==================== end of episode ==================
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 4
current state: 4 action: 0  new state 4
current state: 4 action: 0  new state 4
current state: 4 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 4
current state: 4 action: 0  new state 8
==========

current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 4
current state: 4 action: 0  new state 8
current state: 8 action: 0  new state 8
current state: 8 action: 0  new state 8
==================== end of episode ==================
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 4
current state: 4 action: 0  new state 4
current state: 4 action: 0  new state 8
current state: 8 action: 0  new state 4
current state: 4 action: 0  new state 8
current state: 8 action: 0  new state 4
current state: 4 action: 0  new state 8
current state: 8 action: 0  new state 4
current state: 4 action: 0  new state 4
current state: 4 action: 0  new state 0
current state: 0 action: 0  new state 4
current state: 4 action: 0  new state 8
==================== end of episode ==================
current state: 0 action: 0  new state 0
current st

current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 4
current state: 4 action: 0  new state 4
current state: 4 action: 0  new state 8
current state: 8 action: 0  new state 8
==================== end of episode ==================
current state: 0 action: 0  new state 4
current state: 4 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 4
current state: 4 action: 0  new state 8
==================== end of episode ==================
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 4
current state: 4 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 4
current state: 4 action: 0  new state 4
current state: 4 action: 0  new state 8
==========

current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 4
current state: 4 action: 0  new state 4
current state: 4 action: 0  new state 4
current state: 4 action: 0  new state 4
current state: 4 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 4
current state: 4 action: 0  new state 4
current state: 4 action: 0  new state 4
current state: 4 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 4
current state: 4 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 0
current state: 0 action: 0  new state 4
current state: 4 action: 0  new state 4


current state: 0 action: 3  new state 0
current state: 0 action: 3  new state 1
current state: 1 action: 3  new state 0
current state: 0 action: 3  new state 0
current state: 0 action: 3  new state 0
current state: 0 action: 3  new state 0
current state: 0 action: 3  new state 1
current state: 1 action: 3  new state 2
current state: 2 action: 2  new state 2
current state: 2 action: 2  new state 2
current state: 2 action: 2  new state 6
current state: 6 action: 1  new state 10
current state: 10 action: 0  new state 6
current state: 6 action: 1  new state 10
current state: 10 action: 0  new state 14
current state: 14 action: 3  new state 13
==================== end of episode ==================
current state: 0 action: 3  new state 0
current state: 0 action: 3  new state 0
current state: 0 action: 3  new state 0
current state: 0 action: 3  new state 0
current state: 0 action: 3  new state 0
current state: 0 action: 3  new state 0
current state: 0 action: 3  new state 0
current state: 0 a

KeyboardInterrupt: 

In [322]:
TEST_EPISODES = 1000
env = gym.make(ENV_NAME) #Create a new environment
total_reward = 0
step_count = 0
for i in range(TEST_EPISODES):
    rewards, steps = agent.play_episode(env)
    #print('Episode: ',i,' Reward=', reward)
    total_reward += rewards
    step_count += steps
print('Average reward=', total_reward/TEST_EPISODES, ' total steps =', step_count, ' average steps per episode =', step_count/TEST_EPISODES)
    

Average reward= 0.739  total steps = 39990  average steps per episode = 39.99
